# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

68bbbfc4426c4b179e51fcff3914503f


In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.94,75,100,4.74,GS,1734878797
1,1,thompson,55.7435,-97.8558,-13.91,92,100,2.57,CA,1734878533
2,2,colachel,8.1667,77.2500,27.69,85,1,0.89,IN,1734878800
3,3,adamstown,-25.0660,-130.1015,23.50,86,100,8.55,PN,1734878801
4,4,sungai raya,0.7000,108.9000,26.26,85,100,3.43,ID,1734878802


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# city_data_df = pd.DataFrame(city_data)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    color='City',
    tools=['hover'],  # Enable hover tool to display city names
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600    
)

    

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
# filtered_df = city_data_df[
#     (city_data_df["Humidity"] < 40)
# ]
filtered_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,adamstown,-25.0660,-130.1015,23.50,86,100,8.55,PN,1734878801
4,4,sungai raya,0.7000,108.9000,26.26,85,100,3.43,ID,1734878802
5,5,santo antonio do ica,-3.1022,-67.9397,26.12,88,100,1.58,BR,1734878803
13,13,hobyo,5.3505,48.5268,25.14,69,0,7.55,SO,1734878816
14,14,nuku'alofa,-21.1333,-175.2000,25.19,100,100,3.09,TO,1734878698
...,...,...,...,...,...,...,...,...,...,...
493,493,san cristobal,7.7669,-72.2250,22.74,45,96,1.95,VE,1734879449
504,504,port isabel,26.0734,-97.2086,21.15,82,100,3.09,US,1734879464
517,517,salina cruz,16.1667,-95.2000,24.95,58,83,9.14,MX,1734879481
525,525,pacasmayo,-7.4006,-79.5714,21.74,74,12,3.71,PE,1734879493


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
0,grytviken,GS,-54.2811,-36.5092,75,None
1,thompson,CA,55.7435,-97.8558,92,None
2,colachel,IN,8.1667,77.2500,85,None
3,adamstown,PN,-25.0660,-130.1015,86,None
4,sungai raya,ID,0.7000,108.9000,85,None
...,...,...,...,...,...,...
530,spring creek,US,40.7266,-115.5859,55,None
531,sola,VU,-13.8833,167.5500,80,None
532,muswellbrook,AU,-32.2667,150.9000,66,None
533,camabatela,AO,-8.1881,15.3750,94,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "radius":10000,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    hotel_name = row['Hotel']
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
colachel - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
sungai raya - nearest hotel: No hotel found
santo antonio do ica - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
hettange-grande - nearest hotel: Hôtel Foch
port-aux-francais - nearest hotel: Keravel
smithers - nearest hotel: Sunshine Inn Hotel
vadso - nearest hotel: Scandic Vadsø
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
olonkinbyen - nearest hotel: No hotel found
hobyo - nearest hotel: Iftin Hotel Hobyo
nuku'alofa - nearest hotel: City Hotel
novodvinsk - nearest hotel: Уют
palangkaraya - nearest hotel: Swiss-Belhotel
timaru - nearest hotel: The Grosvenor Hotel
papatowai - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
cabras - nearest hotel: Villa Canu
talnakh - nearest hotel: Талнах
nemuro - nearest hotel: お宿エクハシ
deltebre - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,75,None,No hotel found
1,thompson,CA,55.7435,-97.8558,92,None,Thompson Inn
2,colachel,IN,8.1667,77.2500,85,None,No hotel found
3,adamstown,PN,-25.0660,-130.1015,86,None,No hotel found
4,sungai raya,ID,0.7000,108.9000,85,None,No hotel found
...,...,...,...,...,...,...,...
530,spring creek,US,40.7266,-115.5859,55,None,No hotel found
531,sola,VU,-13.8833,167.5500,80,None,No hotel found
532,muswellbrook,AU,-32.2667,150.9000,66,None,No hotel found
533,camabatela,AO,-8.1881,15.3750,94,None,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
print(hotel_df.isnull().sum())

City            0
Country         0
Lat             0
Lng             0
Humidity        0
Hotel         532
Hotel Name      0
dtype: int64


In [45]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    color='City',
    tools=['hover'],  # Enable hover tool to display city names
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale=1, 
    alpha = .5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)